In [26]:
import pandas as pd 
import geopy.distance
import os
import json
import pickle
import uuid
import warnings
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
import boto3
import numpy as np
import pandas as pd
from dateutil import relativedelta
from scipy.stats import randint as sp_randint
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

In [40]:
df=pd.read_csv('train.csv',nrows=30000000)

In [41]:
def dist(lat1,long1,lat2,long2):
    try:
        coords_1 = (long1,lat1)
        coords_2 = (long2,lat2)

        return(geopy.distance.vincenty(coords_1, coords_2).km)
    except:
        return 'bug'

In [42]:
def man_dist(lat1,long1,lat2,long2):
    return dist(lat1,long1,lat2,long1)+ dist(lat1,long1,lat1,long2)

In [43]:
df['distance']=df.apply(lambda x: dist(x.pickup_latitude,x.pickup_longitude,x.dropoff_latitude,x.dropoff_longitude),axis=1)

/Users/nathantoubiana/anaconda3/lib/python3.6/site-packages/geopy/point.py:81: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  UserWarning)


In [44]:
df['mandistance']=df.apply(lambda x: man_dist(x.pickup_latitude,x.pickup_longitude,x.dropoff_latitude,x.dropoff_longitude),axis=1)

/Users/nathantoubiana/anaconda3/lib/python3.6/site-packages/geopy/point.py:81: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  UserWarning)


TypeError: ("unsupported operand type(s) for +: 'float' and 'str'", 'occurred at index 269695')

In [45]:
df[df.distance!='bug']

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,0.412061
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1,4.64623
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,1.0048
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,0.914154
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,1.36616
5,2011-01-06 09:50:45.0000002,12.1,2011-01-06 09:50:45 UTC,-74.000964,40.731630,-73.972892,40.758233,1,3.23846
6,2012-11-20 20:35:00.0000001,7.5,2012-11-20 20:35:00 UTC,-73.980002,40.751662,-73.973802,40.764842,1,0.802404
7,2012-01-04 17:22:00.00000081,16.5,2012-01-04 17:22:00 UTC,-73.951300,40.774138,-73.990095,40.751048,1,4.38797
8,2012-12-03 13:10:00.000000125,9.0,2012-12-03 13:10:00 UTC,-74.006462,40.726713,-73.993078,40.731628,1,1.50141
9,2009-09-02 01:11:00.00000083,8.9,2009-09-02 01:11:00 UTC,-73.980658,40.733873,-73.991540,40.758138,2,1.42611


In [8]:
df=df[df.distance!='bug']

In [278]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,hour,day,month,year,weekday,distance
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:00,-73.844311,40.721319,-73.841610,40.712278,1,17,15,6,2009,0,0.412061
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:00,-74.016048,40.711303,-73.979268,40.782004,1,16,5,1,2010,1,4.64623
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,0,18,8,2011,3,1.0048
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:00,-73.987130,40.733143,-73.991567,40.758092,1,4,21,4,2012,5,0.914154
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,7,9,3,2010,1,1.36616


In [200]:
y = df['fare_amount'].dropna(axis=0, how='any').as_matrix()

In [202]:
X.shape

(9999545, 6)

In [201]:
X = (df[[ 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count','distance']]
                 .dropna(axis=0, how='any')
                 .as_matrix())

In [203]:
clf = RandomForestRegressor(max_depth=15, max_features=5, n_estimators=100,n_jobs=-1)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=400)
clf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features=5, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [204]:
clf.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features=5, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [205]:
df_test=pd.read_csv('test.csv')

In [206]:
def dist(lat1,long1,lat2,long2):
    try:
        coords_1 = (long1,lat1)
        coords_2 = (long2,lat2)

        return(geopy.distance.vincenty(coords_1, coords_2).km)
    except:
        return 'bug'

In [207]:
df_test['distance']=df_test.apply(lambda x: dist(x.pickup_latitude,x.pickup_longitude,x.dropoff_latitude,x.dropoff_longitude),axis=1)

In [208]:
X = (df_test[['pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count','distance']]
                 ._get_numeric_data()
                 .as_matrix())

In [209]:
predicted = clf.predict(X)



In [210]:
df_test['preds']=predicted

In [211]:
df_test_keys=df_test[['key','preds']]

In [212]:
df_final=pd.read_csv('sample_submission.csv')

In [213]:
df_final.fare_amount=df_final.merge(df_test_keys,on='key')['preds']

In [214]:
df_final[['key','fare_amount']].to_csv('results_nathan.csv' ,  index=False)

In [215]:
s = pickle.dumps(clf)
joblib.dump(clf, 'baseline_model.pkl')

['baseline_model.pkl']

MORE FEATURES

In [9]:
df['pickup_datetime'] = df['pickup_datetime'].str.slice(0, 16)
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')

In [10]:
def transform(data):
    # Extract date attributes and then drop the pickup_datetime column
    data['hour'] = data['pickup_datetime'].dt.hour
    data['day'] = data['pickup_datetime'].dt.day
    data['month'] = data['pickup_datetime'].dt.month
    data['year'] = data['pickup_datetime'].dt.year
    data['weekday']=data['pickup_datetime'].dt.weekday
    data = data.drop('pickup_datetime', axis=1)
    return data

In [11]:
df1=transform(df)

In [12]:
y = df1['fare_amount'].dropna(axis=0, how='any').as_matrix()

In [13]:
X = (df1[[ 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count','distance','year','month','day','hour','weekday']]
                 .dropna(axis=0, how='any')
                 .as_matrix())

In [14]:
X.shape

(55421320, 11)

In [15]:
clf = RandomForestRegressor(max_depth=15, max_features=5, n_estimators=100,n_jobs=-1)

In [16]:
clf.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features=5, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [17]:
df_test=pd.read_csv('test.csv')

In [18]:
df_test['distance']=df_test.apply(lambda x: dist(x.pickup_latitude,x.pickup_longitude,x.dropoff_latitude,x.dropoff_longitude),axis=1)

In [19]:
df_test['pickup_datetime'] = df_test['pickup_datetime'].str.slice(0, 16)
df_test['pickup_datetime'] = pd.to_datetime(df_test['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')

In [20]:
df_test=transform(df_test)

In [21]:
X = (df_test[['pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count','distance','year','month','day','hour','weekday']]
                 ._get_numeric_data()
                 .as_matrix())

In [22]:
X.shape

(9914, 11)

In [23]:
predicted = clf.predict(X)



In [24]:
df_test['preds']=predicted
df_test_keys=df_test[['key','preds']]
df_final=pd.read_csv('sample_submission.csv')
df_final.fare_amount=df_final.merge(df_test_keys,on='key')['preds']
df_final[['key','fare_amount']].to_csv('results_nathan.csv' ,  index=False)

In [25]:
s = pickle.dumps(clf)
joblib.dump(clf, 'model_3.pkl')

['model_3.pkl']